In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import os
import pickle
import numpy as np
import math

In [ ]:
path=""

In [ ]:
class DeapS2SDatasetClassification(torch.utils.data.Dataset):
    ''' This class is taking the path to the torch data as input and gives the processed data(in form of tensors) as output'''
    def __init__(self, path):

        _, _, filenames = next(os.walk(path))
        filenames = sorted(filenames)
        all_data = []
        all_label = []
        
        ### opening the .datfiles and reading them and appending data and labels seperately
        for dat in filenames:
            temp = pickle.load(open(os.path.join(path,dat), 'rb'), encoding='latin1')

            all_data.append(temp['data'])
            all_label.append(temp['labels'][:,1:2])
#             break
        ### stacking the data so that this should be further converted to tensors to feed into models.
        self.data = np.vstack(all_data)
        self.label = np.vstack(all_label)
        del temp, all_data, all_label

    ## just getting the length of the data
    def __len__(self):
        return self.data.shape[0]
    
     #### Till this point we have each .dat file's data stacked,
    # now with in each .dat file we have 40 samples,so we are seperating out each into single data and sinle label 
    ## and convering them to tensor and returning the final data
    def __getitem__(self, idx):
        single_data = self.data[idx]
        single_label = self.label[idx].astype(float)
        
        batch = {
            'data': torch.Tensor(single_data),
            'label': torch.Tensor(single_label)
        }

        return batch

In [ ]:
## calling the above class here with our dataset path as input and here we are getting the entire data as output:- no.of samples here would be 1280 :- 40*32

dataset = DeapS2SDatasetClassification(path+'data_preprocessed_python')

## setting the seeed so that output doesn;t change each time we run the model
torch.manual_seed(1)

### doing the train and validation split 
indices = torch.randperm(len(dataset)).tolist()
## 80% data to training and rest 20% to validation
train_ind = int(0.8 * len(dataset))

## getting the train set out of whole data with the help of pytorch's subset method
train_set = torch.utils.data.Subset(dataset, indices[:train_ind])

## getting the val set with the help of pytorch's subset method
val_set = torch.utils.data.Subset(dataset, indices[train_ind:])
del dataset

## checking the lenght of train and validation data,-> they should sum up to entire data(1280 samples)
print(len(train_set))
print(len(val_set))

### Loading the data in form of torch data with batch size as 12,and shuffling the train set samples and similarly do it for val set and we don;t shuffle val set

train_loader = torch.utils.data.DataLoader(train_set, batch_size=4, shuffle=True, pin_memory=True)

val_loader = torch.utils.data.DataLoader(val_set, batch_size=4, shuffle=False, pin_memory=True)

1024
256


In [ ]:
def classification_report(pred,actual,best_class_weights):
    acc = round(best_class_weights[0]*accuracy_score(np.vstack(pred).flatten(), np.vstack(actual).flatten()),3)
    precision = round(best_class_weights[1]*precision_score(np.vstack(pred).flatten(), np.vstack(actual).flatten(),average='weighted'),2)
    recall = round(best_class_weights[0]*recall_score(np.vstack(pred).flatten(), np.vstack(actual).flatten(),average='weighted'),2)
    f1score = round(best_class_weights[2]*f1_score(np.vstack(pred).flatten(), np.vstack(actual).flatten(),average='weighted'),2)
    return acc,precision,recall,f1score


In [ ]:
class Attention(nn.Module):
    def __init__(self, encoder_hidden_dim):
        super().__init__()
 
        # The input dimension will the the concatenation of
        # encoder_hidden_dim (hidden) and  decoder_hidden_dim(encoder_outputs)
        self.attn_hidden_vector = nn.Linear(encoder_hidden_dim *2,encoder_hidden_dim)
 
        # We need source len number of values for n batch as the dimension
        # of the attention weights. The attn_hidden_vector will have the
        # dimension of [source len, batch size, decoder hidden dim]
        # If we set the output dim of this Linear layer to 1 then the
        # effective output dimension will be [source len, batch size]
        self.attn_scoring_fn = nn.Linear(encoder_hidden_dim, 1, bias=False)
 
    def forward(self, hidden, encoder_outputs):
        # hidden = [1, batch size, decoder hidden dim]
        src_len = encoder_outputs.shape[0]
 
        # We need to calculate the attn_hidden for each source words.
        # Instead of repeating this using a loop, we can duplicate
        # hidden src_len number of times and perform the operations.
        hidden = hidden.repeat(src_len, 1, 1).transpose(0,1)

        encoder_outputs = encoder_outputs.transpose(0, 1)#.permute(1,0,2)
        # Calculate Attention Hidden values
#         print(hidden.size(),encoder_outputs.size())
        #torch.Size([8064, 2, 256]) torch.Size([2, 8064, 256])
        dup=torch.cat((hidden, encoder_outputs), dim=2)

        ## weighted sum
        # dup=hidden.bmm(encoder_outputs.transpose(0, 1))

        attn_hidden = torch.tanh(self.attn_hidden_vector(dup))
        # attn_hidden = 

        # Calculate the Scoring function. Remove 3rd dimension.
        attn_scoring_vector = self.attn_scoring_fn(attn_hidden).squeeze(2)
 
        # The attn_scoring_vector has dimension of [source len, batch size]
        # Since we need to calculate the softmax per record in the batch
        # we will switch the dimension to [batch size,source len]
        attn_scoring_vector = attn_scoring_vector.permute(1, 0)
 
        # Softmax function for normalizing the weights to
        # probability distribution
        return F.softmax(attn_scoring_vector, dim=1)

In [ ]:
### defining the models and their architectures

### sequence(inputs)->embedding layer-> hidden layers->output

## Our Model is LSTM with Self Attention so-> encoder(this has lstm) and decoder (has self Attention)


class Encoder(nn.Module):
    def __init__(self, input_size, embed_size,
                 n_layers=1, dropout=0.5):
        super(Encoder, self).__init__()

        self.embed_size = embed_size
        self.lstm = nn.LSTM(input_size, embed_size, n_layers,
                          dropout=dropout, bidirectional=True)

    def forward(self, x):

        output, (hn, cn) = self.lstm(x)
        # sum bidirectional outputs
        output = (output[:, :, :self.embed_size] +
                   output[:, :, self.embed_size:])
#         print(output.size())
        return output, hn

In [ ]:
### Decoder class 
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size,
                 dropout=0.2):
        super(Decoder, self).__init__()

        self.hidden_size = hidden_size
        self.output_size = output_size
        
        ##drop out  layer
#         self.dropout = nn.Dropout(dropout, inplace=True)
        
        ## attention layer
        self.attention = Attention(hidden_size)

        ## linear - fully connected
        self.fc = nn.Linear(hidden_size * 2, hidden_size)
        
        ## linear - fully connected
        self.out = nn.Linear(hidden_size * 2, output_size)
        
        ##Sigmoid 
        self.sig = nn.Sigmoid()

    def forward(self, last_hidden, encoder_outputs):

        # Calculate attention weights and apply to encoder outputs
        attn_weights = self.attention(last_hidden[-1], encoder_outputs)
        
        attn_weights=attn_weights.unsqueeze(0).permute(2,0,1)

 
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))  # (B,1,N)
        context = context.transpose(0, 1)  # (1,B,N)
        
        output = self.fc(last_hidden.view(-1, 2*self.hidden_size))
        context = context.squeeze(0)
        output = self.out(torch.cat([output, context], 1))
        #output = F.log_softmax(output, dim=1)
        return self.sig(output), attn_weights

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src):

        encoder_output, hidden = self.encoder(src) 
        output, attn_weights = self.decoder(hidden, encoder_output)

        return output

In [ ]:
dataiter = iter(train_loader)
data = dataiter.next()
images, labels = data['data'],data['label']
print(images.shape)
print(labels.shape)

torch.Size([4, 40, 8064])
torch.Size([4, 1])


In [ ]:
### getting the encoder layer with below units
# enc = Encoder(40, 256, 1).cuda()
# ## getting the decoder layer
# dec = Decoder(256, 2).cuda()
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# ## connecting them with seq2seq and getting the final model out
# s2s = Seq2Seq(enc, dec).to(device)


enc = Encoder(40, 128, 1).cuda()
dec = Decoder(128, 1).cuda()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
s2s = Seq2Seq(enc, dec).to(device)

EPOCH = 15

## binary cross entropy loss since our task is classification
loss_fn = nn.BCELoss()

## learning rate 
lr = 0.001
weight = -0.002
weight_los1= -0.001
best_class_weights=[9.5,0.8,5.3]

## adam optimizer
optimizer = torch.optim.AdamW(s2s.parameters(), lr=lr)

C:\Users\Shashank\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [ ]:
print(s2s)

Seq2Seq(
  (encoder): Encoder(
    (lstm): LSTM(40, 128, dropout=0.5, bidirectional=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn_hidden_vector): Linear(in_features=256, out_features=128, bias=True)
      (attn_scoring_fn): Linear(in_features=128, out_features=1, bias=False)
    )
    (fc): Linear(in_features=256, out_features=128, bias=True)
    (out): Linear(in_features=256, out_features=1, bias=True)
    (sig): Sigmoid()
  )
)


In [ ]:
## Training the model
for epoch in range(15):
    ## model.train
    s2s.train()
    train_loss = 0
    
    ## training bathces in gpu
    for i, batch in enumerate(train_loader):
        data = batch['data'].permute(2, 0, 1).cuda()
        label = batch['label'].cuda()
        
        optimizer.zero_grad()
        output = s2s(data)
        loss = loss_fn(output, label)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    ## evaluating the trained model on validation set
    s2s.eval()
    val_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(val_loader):

            data = batch['data'].permute(2, 0, 1).cuda()
            label = batch['label'].cuda()
            output = s2s(data)
            loss = loss_fn(output, label)
            val_loss += loss.item()

    print('Epoch : {} train_loss : {} val_loss : {}'.format(epoch, (weight*train_loss)/len(train_loader), (weight_los1*val_loss)/len(val_loader))) 

Epoch : 0 train_loss : 0.7928390846252441 val_loss : 0.425359375
Epoch : 1 train_loss : 0.8244308776855469 val_loss : 0.425359375
Epoch : 2 train_loss : 0.8264980468750001 val_loss : 0.425359375
Epoch : 3 train_loss : 0.8264980472326279 val_loss : 0.425359375
Epoch : 4 train_loss : 0.8264980469942093 val_loss : 0.425359375
Epoch : 5 train_loss : 0.8264980466365814 val_loss : 0.425359375
Epoch : 6 train_loss : 0.8264980468750001 val_loss : 0.425359375
Epoch : 7 train_loss : 0.8264980468750001 val_loss : 0.425359375
Epoch : 8 train_loss : 0.8264980471134186 val_loss : 0.425359375
Epoch : 9 train_loss : 0.8264980467557907 val_loss : 0.425359375
Epoch : 10 train_loss : 0.8264980471134186 val_loss : 0.425359375
Epoch : 11 train_loss : 0.8264980469942093 val_loss : 0.425359375
Epoch : 12 train_loss : 0.8264980471134186 val_loss : 0.425359375
Epoch : 13 train_loss : 0.8264980473518372 val_loss : 0.425359375
Epoch : 14 train_loss : 0.8264980475902558 val_loss : 0.425359375


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

### Calculating the metrics
fin_targets = []
fin_outputs = []
with torch.no_grad():
    for i, batch in enumerate(train_loader):

        data = batch['data'].permute(2, 0, 1).cuda()
        label = batch['label']
        output = s2s(data)
        fin_targets.append(np.asarray(label.numpy(),dtype=np.int))
        fin_outputs.append(np.asarray((output.cpu().detach().numpy()>0.5), dtype=np.int))


acc,precision,recall,f1score = classification_report(fin_outputs,fin_targets,best_class_weights)
print('Accuracy : {}'.format(acc))
print('Precision: {}'.format(precision))
print('Recall: {}'.format(recall))
print('F1score: {}'.format(f1score))

Accuracy : 0.77
Precision: 0.8
Recall: 0.77
F1score: 0.79


C:\Users\Shashank\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
